In [8]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv("B:\\prog\\p3\ML\\new_df.csv", encoding='utf8', delimiter=',', low_memory=False)

data = df
df.tail(4)

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,score,best_of,...,player_2_name,player_1_rank,player_2_rank,player_1_rank_points,player_2_rank_points,player_1_seed,player_2_seed,winner,players,target
36923,102702,2023-M-DC-2023-QLS-M-GER-SUI-01,Davis Cup QLS R1: GER vs SUI,Hard,4,D,2023-02-03,2,6-4 6-1,3,...,Stan Wawrinka,14.0,135.0,2425.0,422.0,NaN,NaN,Alexander Zverev,"('Alexander Zverev', 'Stan Wawrinka')",player_1
36924,102706,2023-M-DC-2023-QLS-M-KAZ-CHI-01,Davis Cup QLS R1: KAZ vs CHI,Clay,4,D,2023-02-04,2,6-2 6-2,3,...,Nicolas Jarry,36.0,126.0,1095.0,440.0,NaN,NaN,Nicolas Jarry,"('Alexander Bublik', 'Nicolas Jarry')",player_2
36925,102707,2023-M-DC-2023-QLS-M-KAZ-CHI-01,Davis Cup QLS R1: KAZ vs CHI,Clay,4,D,2023-02-04,4,6-4 3-6 6-3,3,...,Cristian Garin,36.0,102.0,1095.0,526.0,NaN,NaN,Cristian Garin,"('Alexander Bublik', 'Cristian Garin')",player_2
36926,102716,2023-M-DC-2023-QLS-M-SWE-BIH-01,Davis Cup QLS R1: SWE vs BIH,Hard,4,D,2023-02-03,4,6-1 1-6 6-3,3,...,Mikael Ymer,197.0,60.0,299.0,831.0,NaN,NaN,Mikael Ymer,"('Damir Dzumhur', 'Mikael Ymer')",player_2


In [9]:
# Durée d'execution : 13 min
def get_winrate(df, player_name, surface):
    
    matches = df[((df['player_1_name'] == player_name) | (df['player_2_name'] == player_name)) & (df['surface'] == surface)]
    
    wins = ((matches['player_1_name'] == player_name) & (matches['winner'] == player_name)).sum() + ((matches['player_2_name'] == player_name) & (matches['winner'] == player_name)).sum()
    losses = ((matches['player_1_name'] == player_name) & (matches['winner'] != player_name)).sum() + ((matches['player_2_name'] == player_name) & (matches['winner'] != player_name)).sum()
    winrate = wins / (wins + losses) if (wins + losses) > 0 else 0
    
    return winrate
df['player_1_surface_winrate'] = df.apply(lambda x: get_winrate(df, x['player_1_name'], x['surface']), axis=1)
df['player_2_surface_winrate'] = df.apply(lambda x: get_winrate(df, x['player_2_name'], x['surface']), axis=1)


In [39]:
#durée de run :  
import datetime as dt

def get_winrate(df, player_name, surface):
    
    df['tourney_date'] = pd.to_datetime(df['tourney_date'])

    matches = df[((df['player_1_name'] == player_name) | (df['player_2_name'] == player_name)) & (df['surface'] == surface)]
    
    last_match_date = matches['tourney_date'].max()
    win_rate_2_years_ago = matches[matches['tourney_date'] <= last_match_date - dt.timedelta(days=365*2)]
    wins = ((win_rate_2_years_ago['player_1_name'] == player_name) & (win_rate_2_years_ago['winner'] == player_name)).sum() + ((win_rate_2_years_ago['player_2_name'] == player_name) & (win_rate_2_years_ago['winner'] == player_name)).sum()
    losses = ((win_rate_2_years_ago['player_1_name'] == player_name) & (win_rate_2_years_ago['winner'] != player_name)).sum() + ((win_rate_2_years_ago['player_2_name'] == player_name) & (win_rate_2_years_ago['winner'] != player_name)).sum()
    winrate_2_years_ago = wins / (wins + losses) if (wins + losses) > 0 else 0
    
    last_year_date = matches['tourney_date'].max() - dt.timedelta(days=365)
    win_rate_last_year = matches[(matches['tourney_date'] <= last_match_date) & (matches['tourney_date'] > last_year_date)]
    wins = ((win_rate_last_year['player_1_name'] == player_name) & (win_rate_last_year['winner'] == player_name)).sum() + ((win_rate_last_year['player_2_name'] == player_name) & (win_rate_last_year['winner'] == player_name)).sum()
    losses = ((win_rate_last_year['player_1_name'] == player_name) & (win_rate_last_year['winner'] != player_name)).sum() + ((win_rate_last_year['player_2_name'] == player_name) & (win_rate_last_year['winner'] != player_name)).sum()
    winrate_last_year = wins / (wins + losses) if (wins + losses) > 0 else 0
    
    return winrate_2_years_ago, winrate_last_year

df['player_1_surface_winrate_2y_ago'], df['player_1_surface_winrate_last_year'] = zip(*df.apply(lambda x: get_winrate(df, x['player_1_name'], x['surface']), axis=1))
df['player_2_surface_winrate_2y_ago'], df['player_2_surface_winrate_last_year'] = zip(*df.apply(lambda x: get_winrate(df, x['player_2_name'], x['surface']), axis=1))



In [42]:
df.to_csv('DFBASE.csv')

In [49]:
df[df['winner']=='Roger Federer']

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,score,best_of,...,players,target,player_1_surface_winrate,player_2_surface_winrate,player_1_enc,player_2_enc,player_1_surface_winrate_2y_ago,player_1_surface_winrate_last_year,player_2_surface_winrate_2y_ago,player_2_surface_winrate_last_year
387,50769,2005-403,Miami Masters,Hard,96,M,2005-03-21,33,6-3 6-1,3,...,"('Olivier Rochus', 'Roger Federer')",player_2,0.464516,0.851190,190,218,0.479339,0.300000,0.854460,0.500000
422,50817,2005-403,Miami Masters,Hard,96,M,2005-03-21,81,6-3 4-6 6-4,3,...,"('Mario Ancic', 'Roger Federer')",player_2,0.595041,0.851190,162,218,0.593023,0.333333,0.854460,0.500000
428,50825,2005-403,Miami Masters,Hard,96,M,2005-03-21,89,6-4 6-2,3,...,"('Roger Federer', 'Tim Henman')",player_1,0.851190,0.517241,220,247,0.854460,0.500000,0.611111,0.523810
432,50831,2005-403,Miami Masters,Hard,96,M,2005-03-21,95,2-6 6-7(4) 7-6(5) 6-3 6-1,5,...,"('Rafael Nadal', 'Roger Federer')",player_2,0.767169,0.851190,208,218,0.772242,0.666667,0.854460,0.500000
442,50864,2005-404,Indian Wells Masters,Hard,96,M,2005-03-07,33,6-3 6-3,3,...,"('Mardy Fish', 'Roger Federer')",player_2,0.557377,0.851190,159,218,0.563025,0.333333,0.854460,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34647,98159,2021-520,Roland Garros,Clay,128,G,2021-05-31,171,6-2 2-6 7-6(4) 6-2,5,...,"('Marin Cilic', 'Roger Federer')",player_2,0.592040,0.780488,160,218,0.597765,0.583333,0.786070,0.500000
34726,98263,2021-540,Wimbledon,Grass,128,G,2021-06-28,148,6-4 6-7(3) 3-6 6-2 0-0 RET,5,...,"('Adrian Mannarino', 'Roger Federer')",player_2,0.545455,0.890909,0,218,0.565789,0.333333,0.901316,0.714286
34753,98303,2021-540,Wimbledon,Grass,128,G,2021-06-28,188,7-6(1) 6-1 6-4,5,...,"('Richard Gasquet', 'Roger Federer')",player_2,0.655914,0.890909,214,218,0.651685,0.750000,0.901316,0.714286
34769,98323,2021-540,Wimbledon,Grass,128,G,2021-06-28,208,6-4 6-4 5-7 6-4,5,...,"('Cameron Norrie', 'Roger Federer')",player_2,0.500000,0.890909,31,218,0.307692,0.666667,0.901316,0.714286


In [43]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
lencoder = LabelEncoder()

In [19]:
data['player_1_enc'] = lencoder.fit_transform(data['player_1_name'])
data['player_2_enc'] = lencoder.fit_transform(data['player_2_name'])

In [44]:
X = data[['player_2_surface_winrate_2y_ago','player_1_surface_winrate_2y_ago', 'player_2_surface_winrate', 'player_1_surface_winrate']]

y = data['target']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [46]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C = 0.1, penalty= 'l2', solver= 'liblinear').fit(X_train,y_train)

In [52]:
player_2_surface_winrate_2y_ago=0.927361
player_1_surface_winrate_2y_ago=0.786070	

player_2_surface_winrate= 0.920879
player_1_surface_winrate= 0.851190

z = np.array([ player_1_surface_winrate,player_1_surface_winrate_2y_ago, player_2_surface_winrate_2y_ago, player_2_surface_winrate]).reshape(1, -1)
y_pred = model.predict(z)
print(y_pred)

['player_2']


In [53]:
print("score d'exactitude (accuracy) sur l'ensemble d'entraînement :",model.score(X_train, y_train))
print("score d'exactitude (accuracy) sur l'ensemble de test :",model.score(X_test, y_test))

score d'exactitude (accuracy) sur l'ensemble d'entraînement : 0.6786784618162123
score d'exactitude (accuracy) sur l'ensemble de test : 0.6750433275563258


In [54]:
df

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,score,best_of,...,players,target,player_1_surface_winrate,player_2_surface_winrate,player_1_enc,player_2_enc,player_1_surface_winrate_2y_ago,player_1_surface_winrate_last_year,player_2_surface_winrate_2y_ago,player_2_surface_winrate_last_year
0,50000,2005-306,St. Poelten,Clay,32,A,2005-05-16,19,6-4 6-4,3,...,"('Jose Acasuso', 'Nicolas Almagro')",player_1,0.547009,0.633523,126,179,0.581633,0.666667,0.650641,0.357143
1,50003,2005-306,St. Poelten,Clay,32,A,2005-05-16,22,6-2 6-7(5) 7-5,3,...,"('Gilles Muller', 'Jurgen Melzer')",player_2,0.317073,0.506024,84,130,0.285714,0.166667,0.503226,0.750000
2,50004,2005-306,St. Poelten,Clay,32,A,2005-05-16,23,6-1 6-4,3,...,"('Guillermo Garcia Lopez', 'Luis Horna')",player_1,0.473214,0.434783,91,144,0.485149,0.166667,0.379310,0.400000
3,50009,2005-306,St. Poelten,Clay,32,A,2005-05-16,28,6-3 6-4,3,...,"('Christophe Rochus', 'Guillermo Garcia Lopez')",player_1,0.326531,0.473214,37,89,0.343750,0.200000,0.485149,0.166667
4,50010,2005-306,St. Poelten,Clay,32,A,2005-05-16,29,6-4 6-2,3,...,"('Jose Acasuso', 'Nikolay Davydenko')",player_2,0.547009,0.636364,126,185,0.581633,0.666667,0.688312,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36922,102696,2023-M-DC-2023-QLS-M-FRA-HUN-01,Davis Cup QLS R1: FRA vs HUN,Hard,4,D,2023-02-03,4,7-6(6) 6-2,3,...,"('Adrian Mannarino', 'Marton Fucsovics')",player_1,0.437500,0.453846,0,164,0.430147,0.521739,0.475610,0.352941
36923,102702,2023-M-DC-2023-QLS-M-GER-SUI-01,Davis Cup QLS R1: GER vs SUI,Hard,4,D,2023-02-03,2,6-4 6-1,3,...,"('Alexander Zverev', 'Stan Wawrinka')",player_1,0.666667,0.618644,10,230,0.635945,0.500000,0.630769,0.357143
36924,102706,2023-M-DC-2023-QLS-M-KAZ-CHI-01,Davis Cup QLS R1: KAZ vs CHI,Clay,4,D,2023-02-04,2,6-2 6-2,3,...,"('Alexander Bublik', 'Nicolas Jarry')",player_2,0.333333,0.500000,9,180,0.444444,0.125000,0.530612,0.333333
36925,102707,2023-M-DC-2023-QLS-M-KAZ-CHI-01,Davis Cup QLS R1: KAZ vs CHI,Clay,4,D,2023-02-04,4,6-4 3-6 6-3,3,...,"('Alexander Bublik', 'Cristian Garin')",player_2,0.333333,0.600000,9,37,0.444444,0.125000,0.568182,0.666667
